In [1]:
import requests
import json
import os



In [4]:
BASE_URL = "https://storage.googleapis.com/spatial-summarizer-3d-models"

MODELS = [
    'headphones', 'aeroplane', 'apple', 'book', 'brain', 
    'brick_toys', 'bulb', 'calculator', 'car', 'computer',
    'dog', 'door', 'earth', 'house', 'laptop', 'pencil',
    'puzzel', 'shark', 'shoes', 'speaker', 'toys', 'wall_clock'
]

def fetch_model(model_name, save_dir='models'):

    model_dir = os.path.join(save_dir, model_name)
    os.makedirs(model_dir, exist_ok=True)
    
    # Construct URL
    url = f"{BASE_URL}/{model_name}/scene.gltf"
    
    print(f"Fetching {model_name} from {url}...")
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        
        gltf_data = response.json()
        
        output_path = os.path.join(model_dir, 'scene.gltf')
        with open(output_path, 'w') as f:
            json.dump(gltf_data, f, indent=2)
        
        print(f"✓ Successfully saved to {output_path}")
        
        if 'buffers' in gltf_data:
            for i, buffer in enumerate(gltf_data['buffers']):
                if 'uri' in buffer and not buffer['uri'].startswith('data:'):
                    bin_url = f"{BASE_URL}/{model_name}/{buffer['uri']}"
                    print(f"  Fetching binary file: {buffer['uri']}...")
                    
                    bin_response = requests.get(bin_url)
                    bin_response.raise_for_status()
                    
                    bin_path = os.path.join(model_dir, buffer['uri'])
                    with open(bin_path, 'wb') as f:
                        f.write(bin_response.content)
                    
                    print(f"  ✓ Saved {buffer['uri']}")
        
        if 'images' in gltf_data:
            for image in gltf_data['images']:
                if 'uri' in image and not image['uri'].startswith('data:'):
                    
                    texture_url = f"{BASE_URL}/{model_name}/{image['uri']}"
                    print(f"  Fetching texture: {image['uri']}...")
                    
                    try:
                        tex_response = requests.get(texture_url)
                        tex_response.raise_for_status()
                        
                        tex_path = os.path.join(model_dir, image['uri'])
                        os.makedirs(os.path.dirname(tex_path), exist_ok=True)
                        
                        with open(tex_path, 'wb') as f:
                            f.write(tex_response.content)
                        
                        print(f"  ✓ Saved {image['uri']}")
                    except Exception as e:
                        print(f"  ⚠ Warning: Could not fetch texture {image['uri']}: {e}")
        
        return gltf_data
        
    except requests.exceptions.RequestException as e:
        print(f"✗ Error fetching {model_name}: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"✗ Error parsing JSON for {model_name}: {e}")
        return None


def fetch_all_models(save_dir='models'):

    print(f"Fetching all {len(MODELS)} models...\n")
    
    results = {}
    for model_name in MODELS:
        gltf_data = fetch_model(model_name, save_dir)
        results[model_name] = gltf_data
        print()  # Empty line between models
    
    # Summary
    successful = sum(1 for v in results.values() if v is not None)
    print(f"\n{'='*50}")
    print(f"Summary: {successful}/{len(MODELS)} models fetched successfully")
    print(f"{'='*50}")
    
    return results


def get_model_info(model_name):
    """Get information about a specific model without downloading"""
    url = f"{BASE_URL}/{model_name}/scene.gltf"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        gltf_data = response.json()
        
        info = {
            'name': model_name,
            'url': url,
            'asset_version': gltf_data.get('asset', {}).get('version', 'unknown'),
            'num_nodes': len(gltf_data.get('nodes', [])),
            'num_meshes': len(gltf_data.get('meshes', [])),
            'num_materials': len(gltf_data.get('materials', [])),
            'num_textures': len(gltf_data.get('textures', [])),
            'has_animations': len(gltf_data.get('animations', [])) > 0
        }
        
        return info
        
    except Exception as e:
        print(f"Error getting info for {model_name}: {e}")
        return None


if __name__ == "__main__":
    print("Get Car model info")
    info = get_model_info('car')
    if info:
        print(json.dumps(info, indent=2))
    print()
    print("Get Book model info")
    info = get_model_info('book')
    if info:
        print(json.dumps(info, indent=2))

Get Car model info
{
  "name": "car",
  "url": "https://storage.googleapis.com/spatial-summarizer-3d-models/car/scene.gltf",
  "asset_version": "2.0",
  "num_nodes": 6,
  "num_meshes": 4,
  "num_materials": 4,
  "num_textures": 9,
  "has_animations": false
}

Get Book model info
{
  "name": "book",
  "url": "https://storage.googleapis.com/spatial-summarizer-3d-models/book/scene.gltf",
  "asset_version": "2.0",
  "num_nodes": 3,
  "num_meshes": 1,
  "num_materials": 1,
  "num_textures": 4,
  "has_animations": false
}
